In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import gmaps
from pprint import pprint
import requests
import json

import warnings; warnings.simplefilter('ignore')

import pymongo

In [2]:
from config import gmaps_key
gmaps.configure(api_key=gmaps_key)

In [3]:
tour_file = pd.read_csv('tour_schedule.csv')
tour_file

,Tour,City
0,Spring 2007,"Farmington, New Mexico, USA"
1,Spring 2007,"Tucson, Arizona, USA"
2,Spring 2007,"Dallas, Texas, USA"
3,Spring 2007,"Baton Rouge, Louisiana, USA"
4,Spring 2007,"New Orleans, Louisiana, USA"
...,...,...
458,Fall 2019,"Wartenberg, Germany"
459,Spring 2020,"Denver, Colorado, USA"
460,Spring 2020,"Lafayette, Colorado, USA"
461,Spring 2020,"Flagstaff, Arizona, USA"


In [4]:
# Split up into City state and country and also fix casts to be more familiar (B10,A16,B18 etc)
tour_file['State'] = ''
tour_file['Country'] = ''
tour_file['Cast'] = ''
for index, row in tour_file.iterrows():
    splitd = row['City'].split(', ')
    if len(splitd) == 1:
        tour_file.loc[index, 'State'] = '-'
        tour_file.loc[index, 'City'] = splitd[0]
        tour_file.loc[index, 'Country'] = splitd[0]
    elif len(splitd) == 2:
        tour_file.loc[index, 'State'] = '-'
        tour_file.loc[index, 'City'] = splitd[0]
        tour_file.loc[index, 'Country'] = splitd[1]
    elif len(splitd) == 3:
        tour_file.loc[index, 'State'] = splitd[1]
        tour_file.loc[index, 'City'] = splitd[0]
        tour_file.loc[index, 'Country'] = splitd[2]
    splitt = row['Tour'].split(' ')
    if splitt[0] == 'Spring':
        cast = 'A'
    elif splitt[0] == 'Fall':
        cast = 'B'
    tour_file.loc[index, 'Cast'] = cast + splitt[1][-2] + splitt[1][-1] 
tour_file.head()

,Tour,City,State,Country,Cast
0,Spring 2007,Farmington,New Mexico,USA,A07
1,Spring 2007,Tucson,Arizona,USA,A07
2,Spring 2007,Dallas,Texas,USA,A07
3,Spring 2007,Baton Rouge,Louisiana,USA,A07
4,Spring 2007,New Orleans,Louisiana,USA,A07


In [5]:
# Url to connect to google geocode api to retrieve Lat and Long based on city state and country
url = f'https://maps.googleapis.com/maps/api/geocode/json?key={gmaps_key}'
# For loop adds lat and long to each tour stop
for index, row in tour_file.iterrows():
    if row['State'] != '-':
        append_url = f'&address=' + str(row['City']) + '%20' + str(row['State']) + '%20' + str(row['Country'])
    else:
        append_url = f'&address=' + str(row['City']) + '%20' + str(row['Country'])
    response = requests.get(url+append_url).json()
    tour_file.loc[index, 'Lat'] = response['results'][0]['geometry']['location']['lat']
    tour_file.loc[index, 'Long'] = response['results'][0]['geometry']['location']['lng']
    
tour_file.head()

,Tour,City,State,Country,Cast,Lat,Long
0,Spring 2007,Farmington,New Mexico,USA,A07,36.728058,-108.218686
1,Spring 2007,Tucson,Arizona,USA,A07,32.222607,-110.974711
2,Spring 2007,Dallas,Texas,USA,A07,32.776664,-96.796988
3,Spring 2007,Baton Rouge,Louisiana,USA,A07,30.451468,-91.187147
4,Spring 2007,New Orleans,Louisiana,USA,A07,29.951066,-90.071532


In [6]:
tour_file.to_csv('yep.csv')

In [17]:
# Connecting to MongoDB to save dataset
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db = client.uwp_db

tour_schedule = db.tour_schedule

tour_schedule.insert_many(tour_file.to_dict('records'))